# Week 2 -- Convolutions

In [15]:
from prettytable import PrettyTable
import torch.nn as nn
import torch
try:
    import torchvision
except:
    !pip install torchvision
    import torchvision
import torchvision.ops as ops

## used from: https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


### Regular Convolution
    1. Counting number of weights without biases
    2. Counting number of weights with biases

In [16]:

class Model(nn.Module):
    def __init__(self,Cin,Cout,K,S, bias=False, G=1):
        super(Model, self).__init__()
        self.conv = nn.Conv2d(in_channels=Cin, out_channels=Cout, kernel_size=K, stride=S, padding=0, dilation=1, groups=G, bias=bias, padding_mode='zeros')

    def forward(self, x):
        return self.conv(x)

model_no_bias = Model(Cin=10,Cout=20,K=3,S=1, bias=False, G=1)
model_with_bias = Model(Cin=10,Cout=20,K=3,S=1, bias=True, G=1)

print(f"\n****** Weights without Bias ******\n")
count_parameters(model_no_bias)

print(f"\n****** Weights with bias ******\n")
count_parameters(model_with_bias)


****** Weights without Bias ******

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| conv.weight |    1800    |
+-------------+------------+
Total Trainable Params: 1800

****** Weights with bias ******

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| conv.weight |    1800    |
|  conv.bias  |     20     |
+-------------+------------+
Total Trainable Params: 1820


1820

### Dilated Convolution

In [17]:
class DilatedConvModel(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, padding):
        super(DilatedConvModel, self).__init__()
        self.dilated_conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation)

    def forward(self, x):
        return self.dilated_conv(x)

# Example usage
model_regular = DilatedConvModel(in_channels=10, out_channels=20, kernel_size=3, dilation=1, padding=1)
model_dilated = DilatedConvModel(in_channels=10, out_channels=20, kernel_size=3, dilation=2, padding=1)

print(f"\n****** Regular Convolution ******\n")
count_parameters(model_regular)

print(f"\n****** Dilated Convolution ******\n")
count_parameters(model_dilated)


****** Regular Convolution ******

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
| dilated_conv.weight |    1800    |
|  dilated_conv.bias  |     20     |
+---------------------+------------+
Total Trainable Params: 1820

****** Dilated Convolution ******

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
| dilated_conv.weight |    1800    |
|  dilated_conv.bias  |     20     |
+---------------------+------------+
Total Trainable Params: 1820


1820

In [18]:
input = torch.randn(1, 10, 28, 28) ## shape is: (Batch, Channels, Height, Width)

output_regular = model_regular(input)
output_dilated = model_dilated(input)

print(f"\nNon-dilated convolution output shape: {output_regular.shape}")
print(f"\nDilated convolution output shape: {output_dilated.shape}")


Non-dilated convolution output shape: torch.Size([1, 20, 28, 28])

Dilated convolution output shape: torch.Size([1, 20, 26, 26])


### Grouped Convolution

In [19]:
class GroupedConvolution(nn.Module):
    def __init__(self,Cin,Cout,K,S, bias=False, G=1):
        super(GroupedConvolution, self).__init__()
        self.conv = nn.Conv2d(in_channels=Cin, out_channels=Cout, kernel_size=K, stride=S, padding=0, dilation=1, groups=G, bias=bias, padding_mode='zeros')

    def forward(self, x):
        return self.conv(x)

model_2_groups = GroupedConvolution(Cin=10,Cout=20,K=3,S=1, bias=True, G=2)
model_5_groups = GroupedConvolution(Cin=10,Cout=20,K=3,S=1, bias=True, G=5)

print("NOTE: Number of groups (G) should always be a factor of Cin")

print(f"\n****** Weights count for G=2 ******\n")
count_parameters(model_2_groups)

print(f"\n****** Weights count for G=5 ******\n")
count_parameters(model_5_groups)

NOTE: Number of groups (G) should always be a factor of Cin

****** Weights count for G=2 ******

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| conv.weight |    900     |
|  conv.bias  |     20     |
+-------------+------------+
Total Trainable Params: 920

****** Weights count for G=5 ******

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| conv.weight |    360     |
|  conv.bias  |     20     |
+-------------+------------+
Total Trainable Params: 380


380

### Depth-wise convolution

In [20]:
model_depthwise = GroupedConvolution(Cin=10,Cout=10,K=3,S=1, bias=True, G=10)

print(f"\n****** Depth-wise convolution ******\n")
count_parameters(model_depthwise)


****** Depth-wise convolution ******

+-------------+------------+
|   Modules   | Parameters |
+-------------+------------+
| conv.weight |     90     |
|  conv.bias  |     10     |
+-------------+------------+
Total Trainable Params: 100


100

### Depthwise Separable Convolution
    - Involves two separate operations: Depthwise convolution followed by pointwise convolution

In [21]:
class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self,):
        super(DepthwiseSeparableConvolution, self).__init__()
        self.depthwise = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, stride=1, padding=0, dilation=1, groups=10, bias=True, padding_mode='zeros')
        self.pointwise = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

    def forward(self, x):
        return self.pointwise(self.conv(x))

depthwise_separable_conv = DepthwiseSeparableConvolution()
count_parameters(depthwise_separable_conv)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| depthwise.weight |     90     |
|  depthwise.bias  |     10     |
| pointwise.weight |    100     |
|  pointwise.bias  |     10     |
+------------------+------------+
Total Trainable Params: 210


210

### Deformable Convolution

In [22]:
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(DeformableConv2d, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size, kernel_size, stride, padding, dilation, groups, bias=False)

    def forward(self, x):
        offset = self.offset_conv(x)
        return ops.deform_conv2d(x, offset, self.conv.weight, self.conv.bias, self.conv.stride, self.conv.padding, self.conv.dilation)

# Example usage
deform_conv = DeformableConv2d(10, 20, 3, padding=1)
print(f"\n****** Deformable Convolution ******\n")
count_parameters(deform_conv)



****** Deformable Convolution ******

+--------------------+------------+
|      Modules       | Parameters |
+--------------------+------------+
|    conv.weight     |    1800    |
|     conv.bias      |     20     |
| offset_conv.weight |    1620    |
+--------------------+------------+
Total Trainable Params: 3440


3440

In [23]:
# Example input
input_tensor = torch.randn(1, 10, 28, 28)
output = deform_conv(input_tensor)
print(f"\nDeformable convolution output shape: {output.shape}")


Deformable convolution output shape: torch.Size([1, 20, 28, 28])


### Transpose Convolution

In [24]:
class TransposeConvModel(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(TransposeConvModel, self).__init__()
        self.transpose_conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        return self.transpose_conv(x)

# Example usage
model = TransposeConvModel(in_channels=10, out_channels=20, kernel_size=3, stride=2, padding=1)
count_parameters(model)

input = torch.randn(1, 10, 28, 28)
output = model(input)
print(f"\nTranspose convolution output shape: {output.shape}")

+-----------------------+------------+
|        Modules        | Parameters |
+-----------------------+------------+
| transpose_conv.weight |    1800    |
|  transpose_conv.bias  |     20     |
+-----------------------+------------+
Total Trainable Params: 1820

Transpose convolution output shape: torch.Size([1, 20, 55, 55])


### 3D Convolutions

In [25]:
class Model3D(nn.Module):
    def __init__(self):
        super(Model3D, self).__init__()
        self.conv3d = nn.Conv3d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        return self.conv3d(x)

model = Model3D()
count_parameters(model) ## counting: 3*3*3*20 + 20 biases

+---------------+------------+
|    Modules    | Parameters |
+---------------+------------+
| conv3d.weight |    5400    |
|  conv3d.bias  |     20     |
+---------------+------------+
Total Trainable Params: 5420


5420